<a href="https://colab.research.google.com/github/IA2021UR/trabajo-ia-en-la-comunidad-de-la-rioja-equipo-2/blob/main/RESULTADO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Trabajo IA en la comunidad de La Rioja, equipo 2
Dataset: https://unirioja-my.sharepoint.com/:u:/g/personal/joheras_unirioja_es/EbmPj-gacuJMrllR3FsP1IcBSvAh5dDfD7VdZQDkyWjMNA?e=qAKxPT 

##Librerías utilizadas


In [1]:
!pip install pymupdf             #Para el conversor de PDF a Imagen
import fitz                      #Para el conversor de PDF a Imagen
import cv2                       #Para tratamiento de imagenes
import matplotlib.pyplot as plt  #Para mostrar las imagenes


     |████████████████████████████████| 6.4MB 6.8MB/s 


##Conversor de PDF a Imagen

In [5]:
#La función recibe tres strings, convierte el fichero x.pdf en dos imágenes x_0.png y x_1.png (Una por cada página)
def conversorPDF_PNG(nombreFichero, pathOrigen, pathDestino):
  doc = fitz.open(pathOrigen+nombreFichero)
  for pg in range(doc.pageCount):
        page = doc[pg]
        rotate = int(0)
            # El factor de escala para cada tamaño es 2, lo que generará una imagen con cuatro veces la resolución para nosotros.
        zoom_x = 2.0
        zoom_y = 2.0
        trans = fitz.Matrix(zoom_x, zoom_y).preRotate(rotate)
        pm = page.getPixmap(matrix=trans, alpha=False)
        nombreFichero=nombreFichero.split('.')[0]
        name = nombreFichero+"_"+str(pg)+".png"
        pm.writePNG(pathDestino+name)


##Función para alinear imágenes

In [ ]:
def alinea_imagenes(imagen1,template,maxFeatures=500,keepPercent=0.2,debug=False):
  imageGray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
  templateGray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

  orb=cv2.ORB_create(maxFeatures)
  (kpsA,descsA)=orb.detectAndCompute(imageGray,None)
  (kpsB,descsB)=orb.detectAndCompute(templateGray,None)

  method = cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING
  matcher = cv2.DescriptorMatcher_create(method)
  matches = matcher.match(descsA, descsB, None)

  matches = sorted(matches, key=lambda x:x.distance)
  keep = int(len(matches) * keepPercent)
  matches = matches[:keep]

  if debug:
    matchedVis=cv2.drawMatches(image,kpsA,template,kpsB,matches,None)
    matchedVis= imutils.resize(matchedVis,width=1000)
    cv2.waitKey(0)

  ptsA=np.zeros((len(matches),2),dtype="float")
  ptsB=np.zeros((len(matches),2),dtype="float")

  for(i,m) in enumerate(matches):
    ptsA[i]=kpsA[m.queryIdx].pt
    ptsB[i]=kpsA[m.trainIdx].pt

  (H,mask)=cv2.findHomography(ptsA,ptsB,method=cv2.RANSAC)

  (h,w)=template.shape[:2]
  aligned=cv2.warpPerspective(image,H,(w,h))

  return aligned

##Instrucciones


In [7]:
conversorPDF_PNG('fichero.pdf','','')
conversorPDF_PNG('plantilla.pdf','','')

In [8]:
fichero1=cv2.imread("fichero_0.png")
fichero2=cv2.imread("fichero_1.png")

plantilla1=cv2.imread("plantilla_0.png")
plantilla2=cv2.imread("plantilla_1.png")